In [1]:
# orthoFinder parser
# generate the orphan gene from different level

# set up file path
species_file_name_table = 'file_list.csv'
orthogroup_path = 'Orthogroups.tsv'
peptide_file_dir = 'fasta_files'
# set up focal groups in list
focal_list = [
"PdeltoidesWV94_445_v2.1.protein_primaryTranscriptOnly.fa",
"PtremulaxPopulusalbaaltv2.1.primaryTrs.pep.fa",
"PtremulaxPopulusalbav2.1p.primaryTrs.pep.fa",
"Ptrichocarpa_444_v3.1.protein.fa",
"PtrichocarpaStettler14_532_v1.1.protein.fa",
"Ptrichocarpav4.1g.primaryTrs.pep.fa"
]
non_focal_sp = 'Acomosus_321_v3.protein.fa'

# import packages and read in files
import json
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio import SearchIO
from Bio import SeqIO
from pathlib import Path
import argparse
import json
import pandas as pd 
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import itertools
from Bio.SeqFeature import SeqFeature, FeatureLocation

# read files
orthogroup = pd.read_csv(orthogroup_path, delimiter='\t')
species_csv = pd.read_csv(species_file_name_table)

/opt/homebrew/anaconda3/envs/binf36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,63) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# read fasta files
from multiprocessing import Pool
peptide_file_dir = Path.cwd() / Path(peptide_file_dir)

print("reading fasta files")
def read_fasta(file_path, species):
    seq_dict = SeqIO.to_dict(SeqIO.parse(file_path, "fasta"))
    return seq_dict

# pool=Pool(species_csv.shape[0])
pool=Pool()
seq_dict_list=pool.starmap(read_fasta, [(peptide_file_dir / row['file_name.fa'], row['file_name.fa']) for index, row in species_csv.iterrows()])
pool.close()
species_pep_dict = {}
for sn,seq_dict in enumerate(seq_dict_list):
    # print(species_csv.iloc[0].species_id)
    key = species_csv.iloc[0].species_id
    species_pep_dict[key] = seq_dict
species_pep_dict


In [5]:
# read fasta files
peptide_file_dir = Path.cwd() / Path(peptide_file_dir)
len_list = []
for (file_path, species) in tqdm([(peptide_file_dir / row['file_name.fa'], row['file_name.fa']) for index, row in species_csv.iterrows()]):
    len_list.append(read_fasta(file_path, species))
    


100%|██████████| 65/65 [01:43<00:00,  1.60s/it]


In [6]:
# read fasta files
peptide_file_dir = Path.cwd() / Path(peptide_file_dir)
species_pep_dict = {}
print("reading fasta files")
for index, row in tqdm(species_csv.iterrows()):
    species = row['species_id']
    pep_seq_file = peptide_file_dir / row['file_name.fa']
    seq_dict = SeqIO.to_dict(SeqIO.parse(pep_seq_file, "fasta"))
    species_pep_dict[species] = seq_dict

orthogroup = orthogroup.set_index("Orthogroup")

# 108471

0it [00:00, ?it/s]

reading fasta files


65it [00:59,  1.10it/s]


In [3]:

# helper functions
def s2l(species_id):
    return species_csv.set_index('species_id').loc[species_id]['file_name.fa'][:-3]


In [11]:
## start parsing
orphan_dict = {}
subset_dict = {}
subset_sn = 0
count = 0
annotation_table_list = []


print("start parsing")
for L in tqdm(range(0, len(focal_list)+1)):
    for subset in itertools.combinations(focal_list, L):
        subset_sn += 1
        # skip the empty set
        if len(subset) == 0: continue
        # trim OG containing non focal species
        # prepare a total_filter that is defo not in the focal list
        total_filter = orthogroup[s2l(non_focal_sp)].isnull()
        for species_id in species_csv.set_index('species_id').index:
            # OG have focal sp not null
            if species_id in subset:
                total_filter = total_filter & orthogroup[s2l(species_id)].notnull()
                continue
            # every not focal species should be mull
            species_filter = orthogroup[s2l(species_id)].isnull()
            total_filter = total_filter & species_filter
            
        current_table = orthogroup[total_filter]
        orphan_dict[subset] = current_table.dropna(axis=1)
        count += current_table.shape[0]
        # print(f"{list(subset)}: {current_table.shape[0]}")

        gene_list = []
        reference_list = []
        seq_record_list = []
        subset_str = ""
        # prepare annotation table and output fasta file
        for sp in subset:
            subset_str = subset_str + sp + "_"
            for orphan_list in [ og.split(', ') for og in current_table[s2l(sp)] ]:
                for orphan in orphan_list:
                    gene_list.append(orphan)
                    reference_list.append(sp)
                    seq_dict = species_pep_dict[sp]
                    seq_record_list.append(seq_dict[orphan])
        file_name = f"orphan_{subset_sn}_{len(subset)}_{len(gene_list)}.fasta"
        SeqIO.write(seq_record_list, file_name, "fasta")
        subset_dict[subset_sn] = subset

with open('subset_info.json', 'w') as fp:
    json.dump(subset_dict, fp)
print(count)






  0%|          | 0/7 [00:00<?, ?it/s]

start parsing


100%|██████████| 7/7 [00:12<00:00,  1.82s/it]

5518


In [25]:
%%time

gene_list = []

reference_list = []
level_list = []
orthogroup_list = []
peptide_length_list = []
peptide_list = []
isoelectric_point_list = []
molecular_weight_list = []
aromaticity_list = []
instability_index_list = []
flexibility_list = []
secondary_structure_fraction_list = []

bio_embeddings_subcellular_location_list = []
bio_embeddings_membrane_soluble_list = []
bio_embeddings_disorder_list = []

for combination, current_table in tqdm(orphan_dict.items()):
    for sp in combination:
        for (orthogroup,orphan_list) in [ (orthogroup,orphan_genes.split(', '))  for orthogroup, orphan_genes in current_table[s2l(sp)].iteritems()]:
            for orphan in orphan_list:
                gene_list.append(orphan)
                reference_list.append(sp)
                level_list.append(combination)
                orthogroup_list.append(orthogroup)
                # get peptide sequence
                pep_seq = species_pep_dict[sp][orphan]
                analysed_seq = ProteinAnalysis(str(pep_seq.seq).strip('*'))
                peptide_length_list.append(len(str(pep_seq.seq).strip('*')))
                peptide_list.append(pep_seq.seq)
                isoelectric_point_list.append(analysed_seq.isoelectric_point())
                molecular_weight_list.append(analysed_seq.molecular_weight())
                aromaticity_list.append(analysed_seq.aromaticity())
                instability_index_list.append(analysed_seq.instability_index())
                flexibility_list.append(analysed_seq.flexibility())
                secondary_structure_fraction_list.append(analysed_seq.secondary_structure_fraction())
                # TODO: add bio_embeddings
                # bio_embeddings_subcellular_location_list.append(bio_embeddings_dict[sp][bio_embeddings_dict[species_4]['original_id'] == gene_id]['subcellular_location'].values)
                # bio_embeddings_membrane_soluble_list.append(bio_embeddings_dict[sp][bio_embeddings_dict[species_4]['original_id'] == gene_id]['membrane_or_soluble'].values)



big_table = pd.DataFrame.from_dict({
    "gene_id":gene_list,
    "reference": reference_list,
    "orphan_level": level_list,
    "orthogroup": orthogroup_list,
    "peptide_length": peptide_length_list,
    "peptide_sequence": peptide_list,
    "isoelectric_point": isoelectric_point_list,
    "molecular_weight": molecular_weight_list,
    # "CCTOP": cctop_list,
    "aromaticity":aromaticity_list,
    "instability_index":instability_index_list,
    "flexibility":flexibility_list,
    "secondary_structure_fraction([Helix, Turn, Sheet])":secondary_structure_fraction_list,
    # "bio_embeddings_subcellular_location": bio_embeddings_subcellular_location_list,
    # "bio_embeddings_membrane_soluble": bio_embeddings_membrane_soluble_list
})
big_table.to_csv("orphan_info_WIP0717.csv")

100%|██████████| 63/63 [00:11<00:00,  5.61it/s]


CPU times: user 13.2 s, sys: 126 ms, total: 13.3 s
Wall time: 13.4 s


In [ ]:
# species_pep_dict['717m'].keys()
examine_list = []
for sn, gene in enumerate(gene_list):
    
    sp = reference_list[sn]
    seq_dict = species_pep_dict[sp]

    # print(seq_dict[gene])
    examine_list.append(seq_dict[gene])


In [ ]:
from Bio import SeqIO
SeqIO.write(examine_list[0:4000], "orphans_for_blast_1.fasta", "fasta")
SeqIO.write(examine_list[4000:], "orphans_for_blast_2.fasta", "fasta")

3305